# Phase 2: Feature Engineering

## Objective
Transform the synthetic Aadhaar-like dataset from Phase 1 into a district-month
level feature table that captures migration, growth, and digital access signals.

## Input
- `data/raw/aadhaar_events_monthly.csv` (district × month × age_group)

## Output
- `data/processed/district_features.csv` (district × month)

## Engineered Features
| Feature | Description |
|---------|-------------|
| `address_velocity` | Month-over-month change in address updates |
| `migration_index` | Ratio of address updates to total Aadhaar population |
| `youth_ratio` | Proportion of 18-35 age group in population |
| `digital_gap` | Gap between address and mobile update rates |
| `growth_3m_consistent` | True if address updates grew for 3 consecutive months |

## Cell 1: Import Libraries and Set Paths

In [1]:
"""
Import Required Libraries
-------------------------
- pandas: Data manipulation and aggregation
- pathlib: Cross-platform file path handling
"""
import pandas as pd
from pathlib import Path

# Define project paths
PROJECT_ROOT = Path.cwd().parent
RAW_DATA_PATH = PROJECT_ROOT / "data" / "raw" / "aadhaar_events_monthly.csv"
PROCESSED_DATA_PATH = PROJECT_ROOT / "data" / "processed" / "district_features.csv"

# Ensure output directory exists
PROCESSED_DATA_PATH.parent.mkdir(parents=True, exist_ok=True)

print(f"✅ Libraries imported")
print(f"📂 Input: {RAW_DATA_PATH}")
print(f"📂 Output: {PROCESSED_DATA_PATH}")

✅ Libraries imported
📂 Input: d:\Projects\ML - DataScience\Saarthi-Net-Data-Pipeline\data\raw\aadhaar_events_monthly.csv
📂 Output: d:\Projects\ML - DataScience\Saarthi-Net-Data-Pipeline\data\processed\district_features.csv


## Cell 2: Load and Inspect Raw Data

In [2]:
"""
Load Raw Data from Phase 1
--------------------------
Each row represents: district × month × age_group
We need to aggregate this to: district × month
"""
df_raw = pd.read_csv(RAW_DATA_PATH)

print(f"✅ Loaded {len(df_raw):,} rows from Phase 1 output")
print(f"\n📊 Columns: {list(df_raw.columns)}")
print(f"\n📋 Age groups: {df_raw['age_group'].unique().tolist()}")
print(f"📋 Districts: {df_raw['district_id'].nunique()}")
print(f"📋 Months: {df_raw['month'].nunique()}")
print(f"\n📋 Sample rows:")
df_raw.head(10)

✅ Loaded 1,200 rows from Phase 1 output

📊 Columns: ['district_id', 'month', 'age_group', 'address_updates', 'mobile_updates', 'total_aadhaar']

📋 Age groups: ['0-17', '18-25', '26-35', '36-60', '60+']
📋 Districts: 20
📋 Months: 12

📋 Sample rows:


,district_id,month,age_group,address_updates,mobile_updates,total_aadhaar
0,GJ_AMD,2024-01,0-17,363,310,48482
1,GJ_AMD,2024-01,18-25,1762,1504,35270
2,GJ_AMD,2024-01,26-35,2082,1778,46305
3,GJ_AMD,2024-01,36-60,1295,1105,51812
4,GJ_AMD,2024-01,60+,200,170,20159
5,GJ_AMD,2024-02,0-17,580,466,48482
6,GJ_AMD,2024-02,18-25,2817,2264,35270
7,GJ_AMD,2024-02,26-35,3330,2676,46305
8,GJ_AMD,2024-02,36-60,2070,1663,51812
9,GJ_AMD,2024-02,60+,322,258,20159


## Cell 3: Aggregate to District-Month Level

Aggregation rules:
- **Sum** `address_updates` and `mobile_updates` across age groups
- **Max** `total_aadhaar` (since it's the same for each age group within district-month)
- **Sum** youth population (18-25 + 26-35 age groups only)

In [3]:
"""
Aggregate Data to District-Month Level
--------------------------------------
Step 1: Sum address_updates and mobile_updates across all age groups
Step 2: Sum total_aadhaar to get total population (each age group has its portion)
Step 3: Calculate youth population (18-25 + 26-35 only)
"""

# Step 1: Aggregate totals by district-month
df_agg = df_raw.groupby(['district_id', 'month']).agg({
    'address_updates': 'sum',      # Total address updates across all age groups
    'mobile_updates': 'sum',       # Total mobile updates across all age groups
    'total_aadhaar': 'sum'         # Sum of population across all age groups
}).reset_index()

# Step 2: Calculate youth population (18-25 and 26-35 age groups)
# Filter for youth age groups only, then sum their Aadhaar counts
youth_groups = ['18-25', '26-35']
df_youth = df_raw[df_raw['age_group'].isin(youth_groups)].groupby(
    ['district_id', 'month']
)['total_aadhaar'].sum().reset_index()
df_youth.columns = ['district_id', 'month', 'youth_aadhaar']

# Step 3: Merge youth counts into main aggregated dataframe
df_agg = df_agg.merge(df_youth, on=['district_id', 'month'], how='left')

# Sort by district and month for consistent ordering (required for delta calculations)
df_agg = df_agg.sort_values(['district_id', 'month']).reset_index(drop=True)

print(f"✅ Aggregated to {len(df_agg):,} district-month rows")
print(f"\n📊 Columns: {list(df_agg.columns)}")
print(f"\n📋 Sample aggregated rows:")
df_agg.head(10)

✅ Aggregated to 240 district-month rows

📊 Columns: ['district_id', 'month', 'address_updates', 'mobile_updates', 'total_aadhaar', 'youth_aadhaar']

📋 Sample aggregated rows:


,district_id,month,address_updates,mobile_updates,total_aadhaar,youth_aadhaar
0,GJ_AMD,2024-01,5702,4867,202028,81575
1,GJ_AMD,2024-02,9119,7327,202028,81575
2,GJ_AMD,2024-03,7439,5501,202028,81575
3,GJ_AMD,2024-04,6599,5697,202028,81575
4,GJ_AMD,2024-05,11987,10550,202028,81575
5,GJ_AMD,2024-06,10490,9054,202028,81575
6,GJ_AMD,2024-07,7051,5210,202028,81575
7,GJ_AMD,2024-08,9241,8074,202028,81575
8,GJ_AMD,2024-09,10791,8333,202028,81575
9,GJ_AMD,2024-10,5487,4680,202028,81575


## Cell 4: Feature 1 - Address Velocity

**Definition:** Month-over-month percentage change in address updates

**Formula:** `(current_month - previous_month) / previous_month`

**Interpretation:**
- Positive = Increasing migration activity
- Negative = Decreasing migration activity
- 0 = First month (no prior data) or no change

In [4]:
"""
Feature: address_velocity
-------------------------
Measures the rate of change in address updates from one month to the next.
High velocity indicates sudden migration surges (e.g., seasonal labor movement).

Formula: (current_month_updates - previous_month_updates) / previous_month_updates

Edge cases:
- First month for each district: Set to 0 (no prior data)
- Division by zero: Set to 0 (if previous month had 0 updates)
"""

# Calculate previous month's address updates for each district
df_agg['prev_address_updates'] = df_agg.groupby('district_id')['address_updates'].shift(1)

# Calculate velocity: (current - previous) / previous
# Use safe division to avoid divide-by-zero
df_agg['address_velocity'] = (
    (df_agg['address_updates'] - df_agg['prev_address_updates']) / 
    df_agg['prev_address_updates'].replace(0, float('inf'))  # Avoid div by zero
)

# Handle edge cases:
# - First month (NaN from shift): Set to 0
# - Division by zero (inf): Set to 0
df_agg['address_velocity'] = df_agg['address_velocity'].replace([float('inf'), float('-inf')], 0)
df_agg['address_velocity'] = df_agg['address_velocity'].fillna(0)

# Round to 4 decimal places for cleaner output
df_agg['address_velocity'] = df_agg['address_velocity'].round(4)

print(f"✅ address_velocity calculated")
print(f"\n📊 Statistics:")
print(f"   Min: {df_agg['address_velocity'].min():.4f}")
print(f"   Max: {df_agg['address_velocity'].max():.4f}")
print(f"   Mean: {df_agg['address_velocity'].mean():.4f}")
print(f"\n📋 Sample (first district, multiple months):")
df_agg[['district_id', 'month', 'address_updates', 'prev_address_updates', 'address_velocity']].head(6)

✅ address_velocity calculated

📊 Statistics:
   Min: -0.5745
   Max: 1.7011
   Mean: 0.0637

📋 Sample (first district, multiple months):


,district_id,month,address_updates,prev_address_updates,address_velocity
0,GJ_AMD,2024-01,5702,NaN,0.0000
1,GJ_AMD,2024-02,9119,5702.0,0.5993
2,GJ_AMD,2024-03,7439,9119.0,-0.1842
3,GJ_AMD,2024-04,6599,7439.0,-0.1129
4,GJ_AMD,2024-05,11987,6599.0,0.8165
5,GJ_AMD,2024-06,10490,11987.0,-0.1249


## Cell 5: Feature 2 - Migration Index

**Definition:** Ratio of address updates to total Aadhaar population

**Formula:** `total_address_updates / total_aadhaar`

**Interpretation:**
- Higher = More people changing addresses relative to population
- Indicates internal migration pressure in the district

In [5]:
"""
Feature: migration_index
------------------------
Measures the proportion of the population that updated their address.
A higher index suggests higher internal migration activity.

Formula: total_address_updates / total_aadhaar

Expected range: 0 to 1 (typically 0.01 to 0.15 for realistic data)
"""

# Safe division: avoid divide-by-zero if total_aadhaar is 0
df_agg['migration_index'] = (
    df_agg['address_updates'] / 
    df_agg['total_aadhaar'].replace(0, float('inf'))
)

# Handle edge case: if total_aadhaar was 0, set migration_index to 0
df_agg['migration_index'] = df_agg['migration_index'].replace([float('inf'), float('-inf')], 0)
df_agg['migration_index'] = df_agg['migration_index'].round(4)

print(f"✅ migration_index calculated")
print(f"\n📊 Statistics:")
print(f"   Min: {df_agg['migration_index'].min():.4f}")
print(f"   Max: {df_agg['migration_index'].max():.4f}")
print(f"   Mean: {df_agg['migration_index'].mean():.4f}")
print(f"\n📋 Sample:")
df_agg[['district_id', 'month', 'address_updates', 'total_aadhaar', 'migration_index']].head(6)

✅ migration_index calculated

📊 Statistics:
   Min: 0.0060
   Max: 0.0596
   Mean: 0.0307

📋 Sample:


,district_id,month,address_updates,total_aadhaar,migration_index
0,GJ_AMD,2024-01,5702,202028,0.0282
1,GJ_AMD,2024-02,9119,202028,0.0451
2,GJ_AMD,2024-03,7439,202028,0.0368
3,GJ_AMD,2024-04,6599,202028,0.0327
4,GJ_AMD,2024-05,11987,202028,0.0593
5,GJ_AMD,2024-06,10490,202028,0.0519


## Cell 6: Feature 3 - Youth Ratio

**Definition:** Proportion of 18-35 age group in total population

**Formula:** `(aadhaar_18_25 + aadhaar_26_35) / total_aadhaar`

**Interpretation:**
- Higher = More working-age youth population
- Correlates with workforce mobility and economic migration

In [6]:
"""
Feature: youth_ratio
--------------------
Measures the proportion of the population in the 18-35 age bracket.
Youth populations are more mobile and likely to migrate for jobs/education.

Formula: (aadhaar_18_25 + aadhaar_26_35) / total_aadhaar

Expected range: 0 to 1 (typically 0.3 to 0.5 for most districts)
"""

# Safe division: avoid divide-by-zero
df_agg['youth_ratio'] = (
    df_agg['youth_aadhaar'] / 
    df_agg['total_aadhaar'].replace(0, float('inf'))
)

# Handle edge case
df_agg['youth_ratio'] = df_agg['youth_ratio'].replace([float('inf'), float('-inf')], 0)
df_agg['youth_ratio'] = df_agg['youth_ratio'].round(4)

print(f"✅ youth_ratio calculated")
print(f"\n📊 Statistics:")
print(f"   Min: {df_agg['youth_ratio'].min():.4f}")
print(f"   Max: {df_agg['youth_ratio'].max():.4f}")
print(f"   Mean: {df_agg['youth_ratio'].mean():.4f}")
print(f"\n📋 Sample:")
df_agg[['district_id', 'month', 'youth_aadhaar', 'total_aadhaar', 'youth_ratio']].head(6)

✅ youth_ratio calculated

📊 Statistics:
   Min: 0.3859
   Max: 0.4126
   Mean: 0.3984

📋 Sample:


,district_id,month,youth_aadhaar,total_aadhaar,youth_ratio
0,GJ_AMD,2024-01,81575,202028,0.4038
1,GJ_AMD,2024-02,81575,202028,0.4038
2,GJ_AMD,2024-03,81575,202028,0.4038
3,GJ_AMD,2024-04,81575,202028,0.4038
4,GJ_AMD,2024-05,81575,202028,0.4038
5,GJ_AMD,2024-06,81575,202028,0.4038


## Cell 7: Feature 4 - Digital Gap

**Definition:** Gap between address updates and mobile updates

**Formula:** `(address_updates - mobile_updates) / address_updates`

**Interpretation:**
- Higher = More people updating address but NOT mobile (digital exclusion)
- Lower = Most address updaters also have mobile access
- Indicates digital infrastructure gap in the district

In [7]:
"""
Feature: digital_gap
--------------------
Measures the proportion of address updates without corresponding mobile updates.
A higher gap indicates digital exclusion - people migrating but lacking mobile access.

Formula: (address_updates - mobile_updates) / address_updates

Expected range: 0 to 1
- 0 = Everyone updating address also updates mobile (high digital access)
- 1 = No one updating address has mobile access (complete digital exclusion)
"""

# Safe division: avoid divide-by-zero
df_agg['digital_gap'] = (
    (df_agg['address_updates'] - df_agg['mobile_updates']) / 
    df_agg['address_updates'].replace(0, float('inf'))
)

# Handle edge cases
df_agg['digital_gap'] = df_agg['digital_gap'].replace([float('inf'), float('-inf')], 0)
df_agg['digital_gap'] = df_agg['digital_gap'].fillna(0)
df_agg['digital_gap'] = df_agg['digital_gap'].round(4)

# Ensure digital_gap is between 0 and 1 (clamp negative values to 0)
df_agg['digital_gap'] = df_agg['digital_gap'].clip(lower=0, upper=1)

print(f"✅ digital_gap calculated")
print(f"\n📊 Statistics:")
print(f"   Min: {df_agg['digital_gap'].min():.4f}")
print(f"   Max: {df_agg['digital_gap'].max():.4f}")
print(f"   Mean: {df_agg['digital_gap'].mean():.4f}")
print(f"\n📋 Sample:")
df_agg[['district_id', 'month', 'address_updates', 'mobile_updates', 'digital_gap']].head(6)

✅ digital_gap calculated

📊 Statistics:
   Min: 0.1071
   Max: 0.7030
   Mean: 0.3837

📋 Sample:


,district_id,month,address_updates,mobile_updates,digital_gap
0,GJ_AMD,2024-01,5702,4867,0.1464
1,GJ_AMD,2024-02,9119,7327,0.1965
2,GJ_AMD,2024-03,7439,5501,0.2605
3,GJ_AMD,2024-04,6599,5697,0.1367
4,GJ_AMD,2024-05,11987,10550,0.1199
5,GJ_AMD,2024-06,10490,9054,0.1369


## Cell 8: Feature 5 - Growth 3-Month Consistent

**Definition:** True if address updates increased for 3 consecutive months

**Interpretation:**
- `True` = Sustained growth pattern (potential peri-urbanization)
- `False` = Fluctuating or declining pattern

In [8]:
"""
Feature: growth_3m_consistent
-----------------------------
Boolean flag indicating sustained growth in address updates.
True if address_updates increased for the current month AND the previous 2 months.

Logic:
- Month N > Month N-1 AND Month N-1 > Month N-2 AND Month N-2 > Month N-3
- Requires 3 consecutive months of positive growth

Use case: Identifying peri-urban zones with sustained migration inflow.
"""

# Calculate month-over-month growth (positive = growth)
df_agg['growth_m1'] = df_agg.groupby('district_id')['address_updates'].diff() > 0
df_agg['growth_m2'] = df_agg.groupby('district_id')['growth_m1'].shift(1)
df_agg['growth_m3'] = df_agg.groupby('district_id')['growth_m1'].shift(2)

# 3-month consistent growth: current AND previous 2 months all showed growth
df_agg['growth_3m_consistent'] = (
    df_agg['growth_m1'].fillna(False) & 
    df_agg['growth_m2'].fillna(False) & 
    df_agg['growth_m3'].fillna(False)
)

# Convert to boolean type
df_agg['growth_3m_consistent'] = df_agg['growth_3m_consistent'].astype(bool)

print(f"✅ growth_3m_consistent calculated")
print(f"\n📊 Distribution:")
print(df_agg['growth_3m_consistent'].value_counts())
print(f"\n📋 Sample (showing growth flags):")
df_agg[['district_id', 'month', 'address_updates', 'growth_m1', 'growth_m2', 'growth_m3', 'growth_3m_consistent']].head(12)

✅ growth_3m_consistent calculated

📊 Distribution:
growth_3m_consistent
False    235
True       5
Name: count, dtype: int64

📋 Sample (showing growth flags):


,district_id,month,address_updates,growth_m1,growth_m2,growth_m3,growth_3m_consistent
0,GJ_AMD,2024-01,5702,False,NaN,NaN,False
1,GJ_AMD,2024-02,9119,True,False,NaN,False
2,GJ_AMD,2024-03,7439,False,True,False,False
3,GJ_AMD,2024-04,6599,False,False,True,False
4,GJ_AMD,2024-05,11987,True,False,False,False
5,GJ_AMD,2024-06,10490,False,True,False,False
6,GJ_AMD,2024-07,7051,False,False,True,False
7,GJ_AMD,2024-08,9241,True,False,False,False
8,GJ_AMD,2024-09,10791,True,True,False,False
9,GJ_AMD,2024-10,5487,False,True,True,False


## Cell 9: Prepare Final Output Schema

In [9]:
"""
Prepare Final Output
--------------------
Select only the required columns in the exact schema order.
Drop all intermediate/helper columns.
"""

# Define exact output schema
OUTPUT_COLUMNS = [
    'district_id',
    'month',
    'address_velocity',
    'migration_index',
    'youth_ratio',
    'digital_gap',
    'growth_3m_consistent'
]

# Select only required columns
df_output = df_agg[OUTPUT_COLUMNS].copy()

# Sort by district_id and month for deterministic output
df_output = df_output.sort_values(['district_id', 'month']).reset_index(drop=True)

print(f"✅ Final output prepared: {len(df_output):,} rows")
print(f"\n📊 Columns: {list(df_output.columns)}")
print(f"\n📋 Data types:")
print(df_output.dtypes)
print(f"\n📋 First 10 rows:")
df_output.head(10)

✅ Final output prepared: 240 rows

📊 Columns: ['district_id', 'month', 'address_velocity', 'migration_index', 'youth_ratio', 'digital_gap', 'growth_3m_consistent']

📋 Data types:
district_id              object
month                    object
address_velocity        float64
migration_index         float64
youth_ratio             float64
digital_gap             float64
growth_3m_consistent       bool
dtype: object

📋 First 10 rows:


,district_id,month,address_velocity,migration_index,youth_ratio,digital_gap,growth_3m_consistent
0,GJ_AMD,2024-01,0.0000,0.0282,0.4038,0.1464,False
1,GJ_AMD,2024-02,0.5993,0.0451,0.4038,0.1965,False
2,GJ_AMD,2024-03,-0.1842,0.0368,0.4038,0.2605,False
3,GJ_AMD,2024-04,-0.1129,0.0327,0.4038,0.1367,False
4,GJ_AMD,2024-05,0.8165,0.0593,0.4038,0.1199,False
5,GJ_AMD,2024-06,-0.1249,0.0519,0.4038,0.1369,False
6,GJ_AMD,2024-07,-0.3278,0.0349,0.4038,0.2611,False
7,GJ_AMD,2024-08,0.3106,0.0457,0.4038,0.1263,False
8,GJ_AMD,2024-09,0.1677,0.0534,0.4038,0.2278,False
9,GJ_AMD,2024-10,-0.4915,0.0272,0.4038,0.1471,False


## Cell 10: Data Quality Validation

In [10]:
"""
Data Quality Checks
-------------------
Validate all required constraints before export.
"""

checks_passed = 0
total_checks = 7

# Check 1: No NaN values
nan_count = df_output.isna().sum().sum()
if nan_count == 0:
    print("✅ Check 1: No NaN values")
    checks_passed += 1
else:
    print(f"❌ Check 1: Found {nan_count} NaN values")
    print(df_output.isna().sum())

# Check 2: address_velocity is 0 for first month of each district
first_months = df_output.groupby('district_id').head(1)
first_month_velocity_zero = (first_months['address_velocity'] == 0).all()
if first_month_velocity_zero:
    print("✅ Check 2: address_velocity is 0 for first month")
    checks_passed += 1
else:
    print("❌ Check 2: address_velocity should be 0 for first month")

# Check 3: migration_index between 0 and 1
mi_valid = (df_output['migration_index'] >= 0).all() and (df_output['migration_index'] <= 1).all()
if mi_valid:
    print("✅ Check 3: migration_index in range [0, 1]")
    checks_passed += 1
else:
    print(f"❌ Check 3: migration_index out of range. Min={df_output['migration_index'].min()}, Max={df_output['migration_index'].max()}")

# Check 4: youth_ratio between 0 and 1
yr_valid = (df_output['youth_ratio'] >= 0).all() and (df_output['youth_ratio'] <= 1).all()
if yr_valid:
    print("✅ Check 4: youth_ratio in range [0, 1]")
    checks_passed += 1
else:
    print(f"❌ Check 4: youth_ratio out of range. Min={df_output['youth_ratio'].min()}, Max={df_output['youth_ratio'].max()}")

# Check 5: digital_gap between 0 and 1
dg_valid = (df_output['digital_gap'] >= 0).all() and (df_output['digital_gap'] <= 1).all()
if dg_valid:
    print("✅ Check 5: digital_gap in range [0, 1]")
    checks_passed += 1
else:
    print(f"❌ Check 5: digital_gap out of range. Min={df_output['digital_gap'].min()}, Max={df_output['digital_gap'].max()}")

# Check 6: growth_3m_consistent is boolean
g3m_bool = df_output['growth_3m_consistent'].dtype == bool
if g3m_bool:
    print("✅ Check 6: growth_3m_consistent is boolean type")
    checks_passed += 1
else:
    print(f"❌ Check 6: growth_3m_consistent should be bool, got {df_output['growth_3m_consistent'].dtype}")

# Check 7: Correct row count (20 districts × 12 months = 240)
expected_rows = 20 * 12
if len(df_output) == expected_rows:
    print(f"✅ Check 7: Row count is {expected_rows} (20 districts × 12 months)")
    checks_passed += 1
else:
    print(f"❌ Check 7: Expected {expected_rows} rows, got {len(df_output)}")

print(f"\n{'='*50}")
if checks_passed == total_checks:
    print(f"✅ ALL {total_checks} VALIDATION CHECKS PASSED")
else:
    print(f"⚠️ {checks_passed}/{total_checks} checks passed")

✅ Check 1: No NaN values
✅ Check 2: address_velocity is 0 for first month
✅ Check 3: migration_index in range [0, 1]
✅ Check 4: youth_ratio in range [0, 1]
✅ Check 5: digital_gap in range [0, 1]
✅ Check 6: growth_3m_consistent is boolean type
✅ Check 7: Row count is 240 (20 districts × 12 months)

✅ ALL 7 VALIDATION CHECKS PASSED


## Cell 11: Export to CSV

In [11]:
"""
Export Final Dataset
--------------------
Save the feature-engineered dataset to CSV.
"""

# Export to CSV (no index column)
df_output.to_csv(PROCESSED_DATA_PATH, index=False)

print(f"✅ Dataset exported to: {PROCESSED_DATA_PATH}")
print(f"\n📊 Export Summary:")
print(f"   Total rows: {len(df_output):,}")
print(f"   Districts: {df_output['district_id'].nunique()}")
print(f"   Months: {df_output['month'].nunique()}")
print(f"\n📋 Schema:")
for col in df_output.columns:
    sample_val = df_output[col].iloc[0]
    print(f"   {col}: {df_output[col].dtype} (e.g., {sample_val})")

print(f"\n📄 First 10 rows:")
df_output.head(10)

✅ Dataset exported to: d:\Projects\ML - DataScience\Saarthi-Net-Data-Pipeline\data\processed\district_features.csv

📊 Export Summary:
   Total rows: 240
   Districts: 20
   Months: 12

📋 Schema:
   district_id: object (e.g., GJ_AMD)
   month: object (e.g., 2024-01)
   address_velocity: float64 (e.g., 0.0)
   migration_index: float64 (e.g., 0.0282)
   youth_ratio: float64 (e.g., 0.4038)
   digital_gap: float64 (e.g., 0.1464)
   growth_3m_consistent: bool (e.g., False)

📄 First 10 rows:


,district_id,month,address_velocity,migration_index,youth_ratio,digital_gap,growth_3m_consistent
0,GJ_AMD,2024-01,0.0000,0.0282,0.4038,0.1464,False
1,GJ_AMD,2024-02,0.5993,0.0451,0.4038,0.1965,False
2,GJ_AMD,2024-03,-0.1842,0.0368,0.4038,0.2605,False
3,GJ_AMD,2024-04,-0.1129,0.0327,0.4038,0.1367,False
4,GJ_AMD,2024-05,0.8165,0.0593,0.4038,0.1199,False
5,GJ_AMD,2024-06,-0.1249,0.0519,0.4038,0.1369,False
6,GJ_AMD,2024-07,-0.3278,0.0349,0.4038,0.2611,False
7,GJ_AMD,2024-08,0.3106,0.0457,0.4038,0.1263,False
8,GJ_AMD,2024-09,0.1677,0.0534,0.4038,0.2278,False
9,GJ_AMD,2024-10,-0.4915,0.0272,0.4038,0.1471,False


## Phase 2 Complete

### Output File
`data/processed/district_features.csv`

### Features Generated
| Feature | Type | Description |
|---------|------|-------------|
| `address_velocity` | float | MoM change in address updates |
| `migration_index` | float | Address updates / total population |
| `youth_ratio` | float | Youth (18-35) / total population |
| `digital_gap` | float | (Address - Mobile) / Address updates |
| `growth_3m_consistent` | bool | 3 consecutive months of growth |

### Next Phase
Phase 3 will use these features to compute migration scores and risk labels.